# Run experiments to "Validate Mathis model at Edge, Core and Intermediate Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Setup the scripts and modules

First, we will upload all the scripts we need to analyze the results for this experiment. This needs to be done only once before start of experiment.

In [ ]:
for n in sender_nodes:
    n.upload_file('process_cwn_file.py','process_cwn_file.py')
    n.upload_file('process_iperf_normal.py','process_iperf_normal.py')
    n.upload_file('mathis_sender.py','mathis_sender.py')
    n.upload_file("cwn.sh", "cwn.sh")

Also we need sklearn to process the downloaded files, so install the library in the environment.This needs to be done only once before start of experiment.

In [ ]:
!pip install scikit-learn

## Generate flows

### Set experiment parameters

>cca, delay, test_duration, num_servers, flows, interval, omit

cca is the congestion control algorithm (reno for this experiment)

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

test_duration is the time for which to send the iperf3 flows (10800 is used for these experiments).

num_servers is the number of ports to be opened on each receiver. 
CoreScale : 100,3000 and 500 ports
EdgeScale : 1,3 and 5 ports
IntermediateScale: 10, 30 and 50 ports

flows is the number of parallel flows to be send from each port. It is set to 1 for all the experiments.

interval is the periodic time interval to save the result by iperf3.

omit is the starting n seconds to ignore the iperf values.(set to 0 for all the experiments. Max value=300)

In [ ]:
cca="reno"
delay=200
test_duration=3600
num_servers=100
flows=1
interval=0.01
omit=0

In [ ]:
# generate full factorial experiment
import itertools
exp_factors_core = {
    'scenario': ['core'], 
    'rate': ['10Gbit'],
    'limit': ['375MB'],
    'cca': ["reno"],
    'delay': [20],
    'test_duration': [600],
    'num_servers': [100, 300, 500],
    'flows': [1],
    'interval': [0.01],
    'omit': [0],
    'trial': [1]
}
factor_names = [k for k in exp_factors_core]
factor_lists = list(itertools.product(*exp_factors_core.values()))
exp_lists_core = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

exp_factors_edge = { 
    'scenario': ['edge'], 
    'rate': ['100Mbit'],
    'limit': ['3MB'],
    'cca': ["reno"],
    'delay': [20],
    'test_duration': [600],
    'num_servers': [100, 300, 500],
    'flows': [1],
    'interval': [0.01],
    'omit': [0],
    'trial': [1]
}
factor_names = [k for k in exp_factors_edge]
factor_lists = list(itertools.product(*exp_factors_edge.values()))
exp_lists_edge = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

exp_lists = exp_lists_core + exp_lists_edge

In [ ]:
data_dir = "data/"
# TODO add code: if directory does not exist, make it

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

In [ ]:
import time # to allow resume
import pandas as pd
import numpy as np
import csv
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages
import sys
import os
for exp in exp_lists:

    # Directory for saving the data 
    # Check if directory exists else create it
    directory_name = "data"
    if not os.path.exists(directory_name):
        # Create the directory
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")
    current_working_directory = os.getcwd()
    data_dir = os.path.join(current_working_directory, directory_name)

    # check if we already ran this experiment
    # (allow stop/resume)
    exp_name_str = "_".join( [str(v) for v in exp.values()] )
    c_file_out = data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] ) # file with mathis constant
    j_file_out = data_dir + '/j_' + "_".join( [str(v) for v in exp.values()] ) # file with JFI

    # check if the c file and j file already exist
    if (os.path.exists(c_file_out)) and (os.path.exists(j_file_out)):
        print("Already have " + c_file_out + " and " + j_file_out ", skipping")
    else:
        print("Running experiment to generate " +  c_file_out + " and " + j_file_out )

        # set up edge or core scale setting
        # first delete any existing queue
        router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")
        # then set one up with HW offload
        router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3 offload")
        router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + exp['rate'])
        router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 handle 3: bfifo limit " + exp['limit'])

        # move the experiment exec procedure inside the loop

        ## Get queue statistics on the router before experiment
        router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_before"+exp_name_str+".txt")

        ## Remove existing result files from the hosts #Check if the files are removed from the senders and receivers
        for n in receiver_nodes:
            n.execute("rm -f 60*")
        for n in sender_nodes:
            n.execute("rm -f sender*")
            n.execute("rm -f data*")
            n.execute("rm -f packet*")
            n.execute("rm -f output*")

        #Now set up delay on the receiver interface:
        #First delete any existing queue (don't worry if there is an error, it means there was not!)
        for n in receiver_nodes:
            receiver_inf=n.get_interface(network_name= "link-receiver")
            receiver_inf_name = receiver_inf.get_device_name()
            n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
            n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(exp['delay']) +" ms limit 1000000")
        
        ## Start parallel servers on the receiver
            
        for i, n in enumerate(receiver_nodes):
            n.execute("sudo killall iperf3")
            n.execute_thread(f'chmod +x iperf-parallel-servers.sh && bash iperf-parallel-servers.sh '+str(exp['num_servers']))

        #check if the required number of ports are opened
        for n in receiver_nodes:
            n.execute("ls -1 | wc -l")
        

        ## Start parallel clients on the sender
        for i, n in enumerate(sender_nodes):
            n.execute("sudo killall iperf3")
            n.execute_thread(f'chmod +x iperf-parallel-senders.sh && bash iperf-parallel-senders.sh 10.10.2.1'+str(i)+" "+str(exp['num_servers']+" "+str(exp['test_duration'])+" "+exp['cca']+" "+str(exp['flows'])+" "+str(exp['interval'])+" "+str(exp['omit']))
            n.execute_thread(f'chmod +x cwn.sh && bash cwn.sh 10.10.2.1'+str(i))
        time.sleep(exp['test_duration']+300) 

        ## Get queue statistics on the router after experiment
        router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_before"+exp_name_str+".txt")

        ## Analysing the results 
        # Calculating sum of bandwidth, square of sum of bandwidth, count of flows and jfi and packet drop rate: 

        #To get packet dropped:
        (drop_before,err_drop_before)=router_node.execute("tail --lines=10 tc_before"+exp_name_str+".txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

        #To get packets sent
        (sent_before,err_sent_before)=router_node.execute("tail --lines=10 tc_before"+exp_name_str+".txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

        #To get packet dropped:
        (drop_after,err_drop_after)=router_node.execute("tail --lines=10 tc_after"+exp_name_str+".txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

        #To get packets sent
        (sent_after, err_sent_after)=router_node.execute("tail --lines=10 tc_after"+exp_name_str+".txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

        #Calculate packet drop rate:
        n_seg_dropped=int(drop_after)-int(drop_before)
        n_seg_sent=int(sent_after)-int(sent_before)
        drop_rate=float(n_seg_dropped)/float(n_seg_sent)

        print("Experiment name: " + exp_name_str)
        print("packet drop before running experiment: "+ str(drop_before))
        print("packet sent before running experiment: " + str(sent_before))
        print("packet drop after running experiment: "+ str(drop_after))
        print("packet sent after  running experiment: " + str(sent_after))
        print("packet sent: " + str(n_seg_sent))
        print("packet dropped: " + str(n_seg_dropped))
        print("packet drop rate: " + str(drop_rate))

        #Run the data processing scripts on each sender to get packet loss, congestion window halving events and rtt from iperf3 and ss output.
        for i,n in enumerate(sender_nodes):
            n.execute_thread(f'chmod +x process_cwn_file.py && python3 process_cwn_file.py '+str(i) )
            n.execute_thread(f'chmod +x process_iperf_normal.py && python3 process_iperf_normal.py '+str(i)+" "+str(exp['num_servers'])+" "+str(exp['test_duration'])+" "+exp['cca']+" "+str(exp['flows']))

        #Run mathis_sender.py script on each sender to get packet loss rate and cwnd halving rate of 
        #each flow and save the output to packet_loss_iperf{i}.csv file

        while True:
            i=False
            time.sleep(100)
            for n in sender_nodes:
                (res,err)=n.execute("pgrep -af python")
                if "python3 process_" in res:
                    i=True
            if i:
                continue
            else:
                break

        for i,n in enumerate(sender_nodes):
            n.execute('chmod +x mathis_sender.py && python3 mathis_sender.py '+str(i))

        # Download all the packet_loss_iperf(i).csv file to the environment.
        for i,n in enumerate(sender_nodes):
            n.download_file(data_dir + "/packet_loss"+str(i)+".csv", "/home/ubuntu/packet_loss"+str(i)+".csv")


        ## Process the downloaded file to get the Mathis constant. Final output is saved to output_mathis_C.csv file.
        dat_exp = pd.concat([pd.read_csv("/data_dir/packet_loss"+str(i)+".csv") for i in range(n_endpoints) ], ignore_index=True)
        dat_exp = dat_exp.assign(p_router_drop = n_seg_dropped/n_seg_sent )

        coef_retrans_ss    = LinearRegression(fit_intercept = False).fit(
            ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_ss_retrans'].values) ) ).values.reshape(-1,1), 
            dat_exp['bitrate']*1000.0
        ).coef_
        coef_retrans_iperf = LinearRegression(fit_intercept = False).fit(
            ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_iperf_retrans'].values) ) ).values.reshape(-1,1), 
            dat_exp['bitrate']*1000.0
        ).coef_
        coef_cwnd_halve    = LinearRegression(fit_intercept = False).fit(
            ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_cwnd_halve'].values) ) ).values.reshape(-1,1), 
            dat_exp['bitrate']*1000.0
        ).coef_
        coef_router_dropped = LinearRegression(fit_intercept = False).fit(
            ( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_router_drop'].values) ) ).values.reshape(-1,1), 
            dat_exp['bitrate']*1000.0
        ).coef_
        print(coef_retrans_ss, coef_retrans_iperf, coef_cwnd_halve, coef_router_dropped)

        print( dat_exp.agg({'port': ['count'], 'bitrate': ['sum'], 'data_seg': ['sum'], 'retrans_ss': ['sum'], 'retrans_iperf': ['sum'], 'cwnd_halve': ['sum'], 'rtt': ['mean'] }) )

        p=dat_exp['port'].aggregate('count')
        bw=dat_exp['bitrate'].aggregate('sum')
        seg=dat_exp['data_seg'].aggregate('sum')
        retrans_ss_sum=dat_exp['retrans_ss'].aggregate('sum')
        retrans_iperf_sum=dat_exp['retrans_iperf'].aggregate('sum')
        cwn_halve_sum=dat_exp['cwnd_halve'].aggregate('sum')
        rtt_mean=dat_exp['rtt'].aggregate('mean')

        # write a line to c_file_out
        # write a line to j_file_out
        #c_file_out = data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" # file with mathis constant
        #output_filename= data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" # file with mathis constant
        if not os.path.isfile(c_file_out):
            with open(c_file_out, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                header = 'time_duration', 'ports', 'Base_RTT(ms)', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
                    'total_retransmission_iperf', 'total_retransmission_ss/total_cwnd_half', 'total_retransmission_iperf/total_cwnd_half',\
                    'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped/total_cwnd_half', \
                    'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'
                writer.writerow(header)

        x_retrans_ss=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_ss_retrans'].values) ) ).values.reshape(-1,1)
        x_retrans_iperf=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_iperf_retrans'].values) ) ).values.reshape(-1,1)
        x_cwnd=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_cwnd_halve'].values) ) ).values.reshape(-1,1)
        x_router=( (1448*8*1000)/(dat_exp['rtt']*np.sqrt(dat_exp['p_router_drop'].values) ) ).values.reshape(-1,1)

        predicted_bw_ss    = LinearRegression(fit_intercept = False).fit(x_retrans_ss, dat_exp['bitrate']*1000.0).predict(x_retrans_ss)
        predicted_bw_iperf = LinearRegression(fit_intercept = False).fit(x_retrans_iperf, dat_exp['bitrate']*1000.0).predict(x_retrans_iperf)
        predicted_bw_cwnd    = LinearRegression(fit_intercept = False).fit(x_cwnd, dat_exp['bitrate']*1000.0).predict(x_cwnd)
        predicted_bw_router = LinearRegression(fit_intercept = False).fit(x_router, dat_exp['bitrate']*1000.0).predict(x_router)

        mdape_ss=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_ss)/ (dat_exp['bitrate']*1000.0)))) * 100
        mdape_iperf=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_iperf)/ (dat_exp['bitrate']*1000.0)))) * 100
        mdape_cwnd=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_cwnd)/ (dat_exp['bitrate']*1000.0)))) * 100
        mdape_router=np.median((np.abs(np.subtract(dat_exp['bitrate']*1000.0, predicted_bw_router)/ (dat_exp['bitrate']*1000.0)))) * 100

        with open(c_file_out, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)   
            columns = test_duration, p, delay, bw, seg, cwn_halve_sum, retrans_ss_sum,retrans_iperf_sum, retrans_ss_sum/cwn_halve_sum,\
                retrans_iperf_sum/cwn_halve_sum, coef_retrans_ss[0], coef_retrans_iperf[0], coef_cwnd_halve[0], coef_router_dropped[0],\
                n_seg_dropped, n_seg_sent, n_seg_dropped/cwn_halve_sum, mdape_ss, mdape_iperf,  mdape_cwnd, mdape_router
            writer.writerow(columns)

        #Get JFI and save it to file 'jfi.csv' 
        sq_y_values=(dat_exp['bitrate']*dat_exp['bitrate']).aggregate('sum')
   
        #j_file_out= data_dir + '/j_' + "_".join( [str(v) for v in exp.values()] )+".csv" # file with JFI
        if not os.path.isfile(j_file_out):
            with open(j_file_out, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                header ='CCA', 'Duration of Expt(sec)', 'Base RTT(ms)', 'Total Bandwidth(Kbps)', 'Sum of sq of BW', 'Flow Count', 'JFI'
                writer.writerow(header)
    
        with open(j_file_out, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            columns = cca, test_duration, delay, bw , sq_y_values, p, (bw*bw)/(sq_y_values*p)
            writer.writerow(columns)
          

### Plots 

Installing required packages required for the plots

In [ ]:
!pip install seaborn
!pip install pdfkit
!pip install reportlab

### Table 1

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import Table, TableStyle
import pandas as pd

directory_name = "data"
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

#change the filename, default filename in which the data is stored is "output_mathis_C.csv"
c_files_core = [data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_core]
c_files_edge = [data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_edge]

#c_files_core = ["output_mathis_C.csv" for exp in exp_lists_core]
#c_files_edge = ["output_mathis_C.csv" for exp in exp_lists_core]

#Read each CSV file and append its DataFrame to the list and concatenate it later
# Core 
dfs = []
for filename in c_files_core:
    df = pd.read_csv(filename, header=0,
                      names=['time_duration', 'ports', 'base_rtt', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss_to_total_cwnd_half', 'total_retransmission_iperf_to_total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped_to_total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'])
    dfs.append(df)  
data_core = pd.concat(dfs, ignore_index=True) 
#print(data_core)
#data_core['ports'] = [1000,3000,5000]

dfs = []
for filename in c_files_edge:
    df = pd.read_csv(filename, header=0,
                      names=['time_duration', 'ports', 'base_rtt', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss_to_total_cwnd_half', 'total_retransmission_iperf_to_total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped_to_total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'])
    dfs.append(df)  
data_edge = pd.concat(dfs, ignore_index=True) 

xvals_core=data_core.sort_values(by=['ports'])
xvals_edge = data_edge
packet_loss_C= [xvals_edge['C_router'].aggregate('mean')]
packet_loss=packet_loss_C+xvals_core['C_router'].values.tolist()
flow_num = pd.unique(xvals_core.ports).tolist()
#print(flow_num)
#print(packet_loss)
CWND_half_C= [xvals_edge['C_cwnd'].aggregate('mean')]
data = { 
    "Packet Loss": packet_loss_C+xvals_core['C_router'].values.tolist(),
    "CWND Halving": CWND_half_C+xvals_core['C_cwnd'].values.tolist()
}
df = pd.DataFrame(data)
cols = df.columns.tolist()
for prsn in range(0, df.shape[1]):
    df[cols[prsn]] = df[cols[prsn]].apply(lambda x: f'{"{:.2f}".format(float(x))}')
df = df.T.reset_index()
my_list = [["p(20ms)", 'EdgeScale', "Core Scale Flow Count"]]+ [["Flow"," "]+ flow_num] + df.values.tolist()
table = Table(my_list)
table.setStyle(TableStyle([('SPAN', (0, 0), (0, 1)),
                           (('SPAN', (1, 0), (1, 1))),
                  ('SPAN', (2, 0), (-1, 0)),
                  ('BACKGROUND', (0,0), (-1,0), colors.white),
                  ('BACKGROUND',(0,1),(-1,-1),colors.white),
                  ('BACKGROUND', (0,0), (0,1), colors.white),
                  ('TEXTCOLOR',(0,0),(-1,0),colors.black),
                  ('ALIGN', (0,0), (-1,-1), 'CENTER'),
                  ('FONTNAME', (0,0), (-1,0), 'Helvetica'),
                  ('FONTSIZE', (0,0), (-1,0), 12),
                  ('BOTTOMPADDING', (0,0), (-1,0), 12),
                  ('LINEABOVE', (1, 2), (-1, 2), 1, colors.black),
                  ('GRID',(0, 0), (-1,-1),1,colors.black)]))
pdf_file = 'Table1_Core_20ms.pdf'
c = canvas.Canvas(pdf_file, pagesize=letter)
table.wrapOn(c, inch*7, inch*2)
table.drawOn(c, x=50, y=650)
c.save()



### Fig 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#dict_scale={'Edge':[0,50],'Intermediate':[90,500],'Core':[950,5000]}
#dict_flows={'Edge':[10,30,50],'Intermediate':[100,300,500],'Core':[1000,3000,5000]}
#rtt=[20,100,200]

directory_name = "data"
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

#change the filename, default filename in which the data is stored is "output_mathis_C.csv"
c_files_core = [data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_core]
c_files_edge = [data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_edge]

#c_files_core = ["output_mathis_C.csv" for exp in exp_lists_core]
#c_files_edge = ["output_mathis_C.csv" for exp in exp_lists_core]

#Read each CSV file and append its DataFrame to the list and concatenate it later
# Core 
dfs = []
for filename in c_files_core:
    df = pd.read_csv(filename, header=0,
                      names=['time_duration', 'ports', 'base_rtt', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss_to_total_cwnd_half', 'total_retransmission_iperf_to_total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped_to_total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'])
    dfs.append(df)  
data_core = pd.concat(dfs, ignore_index=True) 
#print(data_core)

# Edge
dfs = []
for filename in c_files_edge:
    df = pd.read_csv(filename, header=0,
                      names=['time_duration', 'ports', 'base_rtt', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss_to_total_cwnd_half', 'total_retransmission_iperf_to_total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped_to_total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'])
    dfs.append(df)  
data_edge = pd.concat(dfs, ignore_index=True) 
#print(data_core)

#data_core['ports'] = [1000,3000,5000]
#data_edge['ports'] = [1000,3000,5000]

#data_edge['mdape_router'] = data_edge['mdape_router']/4
#data_edge['mdape_cwnd'] = data_edge['mdape_cwnd']/4

N = 3
ind = np.arange(N)
width = 0.25
with PdfPages("MedianError_plot.pdf") as pdf:
    plt.rcParams['figure.figsize'] = (5,2)
    fig, ax1 = plt.subplots()
    ax1.set_axisbelow(True)
    ax1.grid()
    xvals=data_core.sort_values(by=['ports'])
    bar1 = ax1.bar(ind, xvals.mdape_router, width, color = 'chocolate')
    bar2 = ax1.bar(ind+width+0.02, xvals.mdape_cwnd, width, color = 'blue')
    port_num_core=pd.unique(xvals.ports)
    
    ax1.set_xlabel("Flow Count")
    ax1.set_ylabel('Error (%)')
    ax1.set_title("Core Scale at Base RTT of 20ms")
    #ax1.set_xticks(ind+0.1, labels=port_num_core)
    ax1.legend( (bar1,bar2), ("Packet Loss Rate", "CWND Halving Rate"), loc='upper left', bbox_to_anchor=(1, 1), frameon=False )
    ax1.set_ylim(0,20)

    xvals_edge = data_edge.sort_values(by=['ports'])
    ax2 = ax1.twiny()
    ax2.tick_params(top=False, right=False, labelright=False, labeltop=False, gridOn=False)
    ax3 = ax2.twinx()
    port_num_edge=pd.unique(xvals_edge.ports)
    print(port_num_edge)
    
    ax3.plot(port_num_edge,xvals_edge.mdape_router, color='red')
    ax3.set_ylim(0,20)
    ax3.legend( ["Home, Packet Loss"], bbox_to_anchor=(1, 0.3),  loc='upper left', frameon=False)
    ax3.tick_params(top=False, right=False, labelright=False, labeltop=False, gridOn=False)
    ax4 = ax2.twinx()
    ax4.plot(port_num_edge,xvals_edge.mdape_cwnd, color='cyan')
    ax4.set_ylim(0,20)
    ax4.legend( ["Home, CWND Halving"], bbox_to_anchor=(1, 0.2),  loc='upper left', frameon=False)
    ax4.tick_params(top=False, right=False, labelright=False, labeltop=False, gridOn=False)
    pdf.savefig(bbox_inches="tight")
    plt.show()
    # plt.close()

### Fig 3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages


directory_name = "data"
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

# List of filenames for core and edge 
c_files_core = [data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_core]
c_files_edge = [data_dir + '/c_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_edge]

#c_files_core = ["output_mathis_C.csv" for exp in exp_lists_core]
#c_files_edge = ["output_mathis_C.csv" for exp in exp_lists_core]

#Read each CSV file and append its DataFrame to the list and concatenate it later
# First we will do the Core 
dfs = []
for filename in c_files_core:
    df = pd.read_csv(filename, header=0, 
                      names=['time_duration', 'ports', 'base_rtt', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss_to_total_cwnd_half', 'total_retransmission_iperf_to_total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped_to_total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'])
    dfs.append(df)  
data_core = pd.concat(dfs, ignore_index=True) 
#print(data_core)

with PdfPages("Fig3_core_Ratio_plot.pdf") as pdf:
        plt.figure()
        plt.rcParams['figure.figsize'] = (5,2)
        plt.rcParams['axes.axisbelow'] = True
        plt.grid()
        Y_val = data_core['router_dropped_to_total_cwnd_half'].tolist()
        X_val = [exp_factors_core['flows'][0]*i for i in exp_factors_core['num_servers']]       
        plt.plot(X_val,Y_val)
        plt.xlabel("Flow Count")
        plt.ylabel('Packet loss to CWND halving rate Ratio', wrap=True)
        plt.title(key+"Scale at base RTT of 20ms")
        plt.ylim(0,max(20,max(Y_val)+2.5))
        plt.yticks(np.arange(0,max(20,max(Y_val)+2.5),5))
        plt.xticks(core_flows)
        pdf.savefig(bbox_inches="tight")
        plt.show()
        plt.close()
        
        
# Now we do for the edge 
dfs = []
for filename in c_files_edge:
    df = pd.read_csv(filename, header=0, 
                      names=['time_duration', 'ports', 'base_rtt', 'BW', 'total_data_seg_out','total_cwnd_half', 'total_retransmission_ss',\
        'total_retransmission_iperf', 'total_retransmission_ss_to_total_cwnd_half', 'total_retransmission_iperf_to_total_cwnd_half',\
        'C_ss', 'C_iperf', 'C_cwnd', 'C_router', 'router_dropped', 'router_sent', 'router_dropped_to_total_cwnd_half', \
        'mdape_ss', 'mdape_iperf', 'mdape_cwnd', 'mdape_router'])
    dfs.append(df)  
data_edge = pd.concat(dfs, ignore_index=True) 
#print(data_core)

with PdfPages("Fig3_edge_Ratio_plot.pdf") as pdf:
        plt.figure()
        plt.rcParams['figure.figsize'] = (5,2)
        plt.rcParams['axes.axisbelow'] = True
        plt.grid()
        Y_val = data_edge['router_dropped_to_total_cwnd_half'].tolist()
        X_val = [exp_factors_edge['flows'][0]*i for i in exp_factors_edge['num_servers']]       
        plt.plot(X_val,Y_val)
        plt.xlabel("Flow Count")
        plt.ylabel('Packet loss to CWND halving rate Ratio', wrap=True)
        plt.title(key+"Scale at base RTT of 20ms")
        plt.ylim(0,max(20,max(Y_val)+2.5))
        plt.yticks(np.arange(0,max(20,max(Y_val)+2.5),5))
        plt.xticks(core_flows)
        pdf.savefig(bbox_inches="tight")
        plt.show()
        plt.close()

### Fig 4 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


N = 3
ind = np.arange(N) 
width = 0.25
color_name = {'20':'royalblue', '100':'tomato', '200':'mediumspringgreen'}
xvals = [0,0,0]
bar = [0,0,0]

directory_name = "data"
current_working_directory = os.getcwd()
data_dir = os.path.join(current_working_directory, directory_name)

# List of filenames for core and edge 
jfi_files_core = [data_dir + '/j_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_core] # file with JFI
jfi_files_edge = [data_dir + '/j_' + "_".join( [str(v) for v in exp.values()] )+".csv" for exp in exp_lists_edge] # file with JFI


#  First let us plot for Core 
#Read each CSV file and append its DataFrame to the list and concatenate it later
dfs = []
for filename in jfi_files_core:
    df  = pd.read_csv(filename, header=0, names=['cca', 'duration','rtt','BW','sq_BW', 'flow_count', 'jfi'])
    dfs.append(df)  
jfi_dat_core = pd.concat(dfs, ignore_index=True) 
cca_core=pd.unique(jfi_dat_core.cca)

with PdfPages("JFI_plot_core.pdf") as pdf:
    for i,c in enumerate(cca_core):
        plt.rcParams['figure.figsize'] = (5,3)
        plt.rcParams['axes.axisbelow'] = True
        plt.grid(axis='y')
        dat_cca=jfi_dat_core[(jfi_dat_core['cca'] == c)]
        dat_cca.sort_values(by=['flow_count'])
        print(dat_cca)
        flows=pd.unique(dat_cca.flow_count)
        rtt_num = pd.unique(dat_cca.rtt)       
        for j,r in enumerate(rtt_num):
            xvals[j] = dat_cca.jfi[jfi_dat_core['rtt']==r]
            bar[j] = plt.bar(ind + (width+0.02)*j, xvals[j], width, color = color_name[str(r)])
            
        plt.xlabel("Flow Count")
        plt.ylabel('JFI')
        plt.title("Core Scale, "+"CCA-"+c)        
        plt.xticks(ind+width,flows)
        plt.legend( (bar[j] for j in range(len(rtt_num))), (str(rtt_num[j]) + " ms" for j in range(len(rtt_num))), bbox_to_anchor=(1, 0.5), frameon=False )   
        #pdf.savefig(bbox_inches="tight")
        plt.show()
        plt.close()
        
#  Now let us plot for Edge 
#Read each CSV file and append its DataFrame to the list and concatenate it later
dfs = []
for filename in jfi_files_edge:
    df  = pd.read_csv(filename, header=0, names=['cca', 'duration','rtt','BW','sq_BW', 'flow_count', 'jfi'])
    dfs.append(df)  
jfi_dat_edge = pd.concat(dfs, ignore_index=True) 
cca_edge=pd.unique(jfi_dat_edge.cca)

with PdfPages("JFI_plot_edge.pdf") as pdf:
    for i,c in enumerate(cca_edge):
        plt.rcParams['figure.figsize'] = (5,3)
        plt.rcParams['axes.axisbelow'] = True
        plt.grid(axis='y')
        dat_cca=jfi_dat[(jfi_dat_edge['cca'] == c)]
        dat_cca.sort_values(by=['flow_count'])
        print(dat_cca)
        flows=pd.unique(dat_cca.flow_count)
        rtt_num = pd.unique(dat_cca.rtt)       
        for j,r in enumerate(rtt_num):
            xvals[j] = dat_cca.jfi[jfi_dat_edge['rtt']==r]
            bar[j] = plt.bar(ind + (width+0.02)*j, xvals[j], width, color = color_name[str(r)])
            
        plt.xlabel("Flow Count")
        plt.ylabel('JFI')
        plt.title("Edge Scale, "+"CCA-"+c)        
        plt.xticks(ind+width,flows)
        plt.legend( (bar[j] for j in range(len(rtt_num))), (str(rtt_num[j]) + " ms" for j in range(len(rtt_num))), bbox_to_anchor=(1, 0.5), frameon=False )   
        #pdf.savefig(bbox_inches="tight")
        plt.show()
        plt.close()


### Save the linear regression plots to a pdf.

This should be placed within the experiment running cell to plot the Linear regression plots. Not required now

In [ ]:
with PdfPages("linear_reg_plot.pdf") as pdf:
  plt.rcParams['figure.figsize'] = (8,6)

  plt.scatter(x=x_retrans_ss, y=dat_exp['bitrate']*1000.0, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x_retrans_ss, y=predicted_bw_ss, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x_retrans_ss, predicted_bw_ss, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)") 
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-1 calculation of packet_loss rate using data_seg_out from ss and retrans from ss data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()

  plt.scatter(x=x_retrans_iperf, y=dat_exp['bitrate']*1000.0, color='C4', s=10, label='actual values')
  plt.scatter(x=x_retrans_iperf, y=predicted_bw_iperf, color='C3', s=10, label='predicted_values')
  plt.plot(x_retrans_iperf, predicted_bw_iperf, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)") 
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-2: calculation of packet_loss rate using data_seg_out from ss and retrans from iperf3 data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()


  plt.scatter(x=x_cwnd, y=dat_exp['bitrate']*1000.0, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x_cwnd, y=predicted_bw_cwnd, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x_cwnd, predicted_bw_cwnd, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)")
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-3: calculation of packet_loss rate using data_seg_out from ss and cwnd from iperf3 data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()

  plt.scatter(x=x_router, y=dat_exp['bitrate']*1000.0, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x_router, y=predicted_bw_router, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x_router, predicted_bw_router, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)")
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-4: calculation of packet_loss rate using packet drop rate at the router")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()
 

Remove the transfered files after data analysis and/or downloading to your local system.

In [ ]:
%rm pac*

In [ ]:
%rm linear*